## for LAND

In [1]:
import pandas as pd

In [2]:
parcels = pd.read_csv('LAND_CustomParcels_Final_2021_Tim.csv')

In [3]:
parcels['test_ownership'] = ''
parcels['test_categories'] = ''

In [ ]:
list(parcels)

In [4]:
categories = pd.read_csv('OwnershipMap.csv')

In [5]:
for index, category in categories.iloc[29:30].iterrows():
    true_owner = category['true_owner[123]'].split('|')
    print(true_owner)

['CHURCH', '&&CHURCH', 'IGLESIA', '&&IGLESIA', 'MINISTERIO', '&&MINISTERIO', '7TH DAY', '&&7TH DAY', 'EGLISE', '&&EGLISE', 'BIBLE', '&&BIBLE', 'ST JOHN', 'CAMP FREEDOM INC', 'TABERNACLE', '&&TABERNACLE', 'BISHOP', '&&BISHOP', 'REVEREND', '&&REVEREND', 'ISLAM', '&&ISLAM', 'TEMPLE', '&&TEMPLE', 'JESUS', '&&JESUS', 'CHAPEL', '&&CHAPEL', 'CER LLC', 'COMMUNITY BLDR & CAPITAL DEV INC', 'LUTHERAN', '&&LUTHERAN', 'NAZARENE', '&&NAZARENE', 'SEVENTH DAY', '&&SEVENTH DAY', 'CHRISTIAN', '&&CHRISTIAN', 'FELLOWSHIP', '&&FELLOWSHIP', 'BAPTIST!HEALTH', '&&BAPTIST!HEALTH', 'JEWISH', '&&JEWISH', 'UPPER ROOM ASSEMBLY INC', 'METHODIST', '&&METHODIST', 'PEDRO SANTANA & MOISES OBREGON&&GUSTAVO FONSECA', 'ADVENTIST', '&&ADVENTIST', 'MT CALVARY COMMUNITY CH', 'MINISTRIES', '&&MINISTRIES', 'PENTECOST', '&&PENTECOST', 'OF FAITH', 'PRESBYTERIAN', '&&PRESBYTERIAN', 'RIVER AND 16TH PROPERTY LLC', 'PRESB CH', '&&PRESB CH', 'WORSHIP', '&&WORSHIP', 'ASSEMBLY OF GOD', '&&ASSEMBLY OF GOD', 'BUDDH', '&&BUDDH', 'AFR METH

In [ ]:
b = ((parcels['true_owner1'].str.contains('FLORIDA CITY') & \
    ~parcels['true_owner1'].str.contains('AGENCY',na=False) & \
    ~parcels['true_owner1'].str.contains('COMMUNITY',na=False) & \
    ~parcels['true_owner1'].str.contains('AGCY',na=False)) & \
    ~parcels['true_owner2'].str.contains('REDEVELOPMENT',na=False) & \
    ~parcels['true_owner2'].str.contains('REDV',na=False))

In [13]:
parcels['test_ownership'] = ''
parcels['test_categories'] = ''
        
def create_condition(owner):
    '''Create a conditional array to filter the parcels dataframe for an ownership
    category using a formatted string from the OwnsershipMap file.'''
    owner = owner.split('&&')
    
    if '!' not in owner[0]: # simple CONTAIN conditional on true_owner1
        #print ('simple','|'+owner[0]+'|')
        lb = parcels['true_owner1'].str.contains(owner[0])
        
    else: # AND NOT CONTAIN conditionals on true_owner1
        owner1s = owner[0].split('!')
        #print ('owner1','|'+owner1s[0]+'|')
        lb = parcels['true_owner1'].str.contains(owner1s[0])
        for owner1 in owner1s[1:]:
            #print ('owner1 not','|'+owner1+'|')
            lb = lb & ~parcels['true_owner1'].str.contains(owner1,na=False)
        lb = (lb)       
    
    if len(owner) > 1:    
        for owner2 in owner[1:]: # AND [NOT] CONTAIN conditionals on true_owner2
            if '!' not in owner2:
                #print ('owner2','|'+owner2+'|')
                lb = lb & parcels['true_owner2'].str.contains(owner2)
            else:
                owner2s = owner2.split('!')
                #print ('owner2','|'+owner2s[0]+'|')
                lb = lb & parcels['true_owner2'].str.contains(owner2s[0],na=False)
                for owner2 in owner2s[1:]:
                    #print ('owner2 not','|'+owner2+'|')
                    lb = lb & ~parcels['true_owner2'].str.contains(owner2,na=False)
        lb = (lb)
            
    return lb

# loop through all the categories in the OwnershipMap file : .iloc[29:30].
for index, category in categories.iterrows():
    true_owner = category['true_owner[123]'].split('|')
    print(category['Ownership'])

    # create the conditional pandas array to filter rows in the parcels
    b = create_condition(true_owner[0])
    for sub_owner in true_owner[1:]:
        b = b | create_condition(sub_owner)
        
    # set the ownership and categories in the parcels with the conditional array
    parcels.loc[b,'test_ownership'] = category['Ownership']
    parcels.loc[b,'test_categories'] = category['Owner Categories']
    
    # give some feedback
    print(
        'original',len(parcels[parcels['Ownership']==category['Ownership']]['Ownership']),
        'test',len(parcels[parcels['test_ownership']==category['Ownership']]['Ownership'])
    )

Baptist Health
original 7 test 7
Barry University
original 15 test 15
City of Aventura
original 1 test 1
City of Coral Gables
original 40 test 40
City of Florida City
original 12 test 0
City of Hialeah
original 12 test 12
City of Hialeah Gardens
original 2 test 2
City of Homestead
original 47 test 21
City of Miami
original 120 test 39
City of Miami Beach
original 110 test 110
City of Miami Gardens
original 17 test 17
City of Miami Springs
original 13 test 13
City of North Bay Village
original 2 test 2
City of North Miami
original 36 test 36
City of North Miami Beach
original 19 test 19
City of Opa Locka
original 30 test 30
City of South Miami
original 1 test 1
City of Sunny Isles Beach
original 3 test 3
City of West Miami
original 3 test 3
Community Redevelopment Agency
original 157 test 157
Expressway Authority
original 26 test 26
Federal
original 31 test 31
Florida International University
original 4 test 4
Florida Memorial University
original 4 test 4
Habitat for Humanity
original 7

In [7]:
parcels[b][['Ownership','true_owner1','true_owner2']]

,Ownership,true_owner1,true_owner2
12,Religious Organization,MANFRED MEMORIAL FOUNDATION INC,% REVEREND GEORGE KELSEY
21,Religious Organization,MINISTERIO INTERNATIONAL EL REY,JESUS INC
26,Religious Organization,UNIVERSITY BAPTIST CHURCH OF,CORAL GABLES INC
27,Religious Organization,CAMP FREEDOM INC,NaN
60,Religious Organization,CANAAN CHURCH OF,HOMESTEAD INC
...,...,...,...
3022,Religious Organization,MT OLIVE MISSIONARY BAPT CHU OF,SOUTH MIAMI FL INC
3027,Religious Organization,GRTR NEW BETHEL BAPTIST CH INC,NaN
3028,Religious Organization,GRTR NEW BETHEL BAPTIST CH INC,NaN
3029,Religious Organization,GRTR NEW BETHEL BAPTIST CH INC,NaN


In [8]:
sub = parcels[b][['ogc_fid','Ownership','true_owner1','true_owner2','dor_code_cur']]

In [9]:
len(sub)

842

In [10]:
sub[sub['Ownership']!='Religious Organization']

,ogc_fid,Ownership,true_owner1,true_owner2,dor_code_cur


In [11]:
org = parcels[parcels['Ownership']=='Religious Organization']

In [12]:
for index, line in org.iterrows():
    if line['ogc_fid'] not in sub['ogc_fid'].values:
        print (line['dor_code_cur'],line['ogc_fid'],line['true_owner1'],line['true_owner2'])

7144 170869 CPC OF WWM USA FOUNDATION INC nan
9981 375622 INTERNATIONAL CENTER OF PRAISE nan
7144 527 P O M INC nan
81 251276 PABLO ALVAREZ WALKIRIA ALVAREZ
2865 141139 SOUTH EAST DISTRICT FLORIDA ANNUAL CONFERENCE
1066 229512 RICHEMOND LAMBERT &W MARIE nan
1081 162004 TRINITY FAITH DEL MIN CENTER INC nan
2865 524028 ANTIOCH MISS BAPT OF CAROL CITY nan
66 239387 HILLEL COMMUNITY DAY SCHOOL INC nan
7144 182351 THE SHUL OF BAL HARBOUR INC nan
2865 112441 DOM INC nan
2865 117877 EDWARD A MCCARTHY nan
81 189510 THE SANCTUARY IN EL PORTAL LP C/O SETH GADINSKY
2865 125456 RINCON DE SAN LAZARO nan
7081 4587 LOUIS M ALEXIS & VEONEL GUERRIER & KELLY LEVASSEUR (TRS)
7081 4586 LOUIS M ALEXIS & VEONEL GUERRIER & KELLY LEVASSEUR (TRS)
81 269200 S EASTERN CONF OF SDA ASSOCIATION nan
7144 7988 MARKS SUBDIVISION APARTMENTS LLC nan
81 3655 TGCG HOLDINGS LLC nan
81 7046 PRAYER & PRAISE INTERNATIONAL nan
2865 143153 NOAH'S ARK PRAYER ROOM INC nan
81 156734 ERSHAD CENTER INC nan
2865 15647 JOHN C FAVALORA